In [148]:
import importlib
import corona_plots
importlib.reload(corona_plots)

corona_plots.CoronaPlots.plot_global_cases(auto_open=True)
#corona_plots.CoronaPlots.plot_usa_cases(auto_open=True)
#corona_plots.CoronaPlots.plot_usa_pos_neg_tracking(auto_open=True)
pass

In [127]:
import hashlib
hashlib.md5(b'12').

'c20ad4d76fe97759aa27a0c99bff6710'

In [131]:
int(hashlib.sha1(b'123').hexdigest(), 16) % (10 ** 8)

83137519

b'123'